## Import

In [1]:
from tqdm import tqdm

import streamlit as st
import pandas as pd
import numpy as np
import OpenDartReader
import warnings
import time
import re, os

warnings.filterwarnings('ignore')

In [5]:
api_key = ''
dart = OpenDartReader(api_key)

### 1. 공시정보 ###

In [7]:
# 특정기업(삼성전자) 특정 날짜 이후 공시목록 (날짜에 다양한 포맷이 가능합니다 2022, '2022-01-01', '20220101' )
dart.list('삼성전자', start='2022-01-01') # 2022-01-01 ~ 오늘

# 특정기업(삼성전자) 특정 일자 범위(start~end)의 공시목록 (날짜에 다양한 포맷이 가능합니다)
dart.list('005930', start='2022-04-28', end='2022-04-28')

# 특정기업(삼성전자) 1999년~이후 모든 정기보고서 (정정된 공시포함)
dart.list('005930', start='1999-01-01', kind='A', final=False)

# 특정기업(삼성전자) 1999년~이후 모든 정기보고서 (최종보고서)
dart.list('005930', start='1999-01-01', kind='A') 

# 2022-07-01 하루동안 모든 기업의 공시목록
dart.list(start='20200701', end='20200701')

# 2022-01-01 ~ 2022-01-10 모든 회사의 모든 공시목록 (3,139 건)
dart.list(start='2022-01-01', end='2022-01-10')

# 2022-01-01 ~ 2022-01-10 모든 회사의 모든 공시목록 (정정된 공시포함) (3,587 건)
dart.list(start='2022-01-01', end='2022-01-10', final=False)

# 2022-01-01~2022-03-30 모든 회사의 정기보고서 (corp를 특정 하지 않으면 최대 3개월) (2,352 건)
dart.list(start='2022-01-01', end='2022-03-30', kind='A')

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00126380,삼성전자,005930,Y,최대주주등소유주식변동신고서,20230503800466,삼성전자,20230503,유
1,00126380,삼성전자,005930,Y,주식등의대량보유상황보고서(일반),20230428001621,삼성물산,20230428,
2,00126380,삼성전자,005930,Y,기타경영사항(자율공시),20230427800505,삼성전자,20230427,유
3,00126380,삼성전자,005930,Y,기타경영사항(자율공시),20230427800501,삼성전자,20230427,유
4,00126380,삼성전자,005930,Y,현금ㆍ현물배당결정,20230427800074,삼성전자,20230427,유
...,...,...,...,...,...,...,...,...,...
294,00126380,삼성전자,005930,Y,임원ㆍ주요주주특정증권등소유상황보고서,20220111000233,김용주,20220111,
295,00126380,삼성전자,005930,Y,임원ㆍ주요주주특정증권등소유상황보고서,20220110000025,정진민,20220110,
296,00126380,삼성전자,005930,Y,임원ㆍ주요주주특정증권등소유상황보고서,20220107000644,고정욱,20220107,
297,00126380,삼성전자,005930,Y,기업설명회(IR)개최(안내공시),20220107800039,삼성전자,20220107,유


In [29]:
dart.list(start = '2023-05-04', end = '2023-05-04')

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,01012640,슈어소프트테크,298830,K,주식등의대량보유상황보고서(일반),20230504000198,배현섭,20230504,
1,01475609,디앤디플랫폼리츠,377190,Y,매출액또는손익구조30%(대규모법인은15%)이상변경,20230504800167,디앤디플랫폼리츠,20230504,유
2,00567222,우림피티에스,101170,K,주식등의대량보유상황보고서(일반),20230504000193,한규석,20230504,
3,00104856,삼성증권,016360,Y,증권발행실적보고서,20230504000190,삼성증권,20230504,
4,00302078,엠피씨플러스,050540,K,주식등의대량보유상황보고서(약식),20230504000183,두레마을,20230504,
...,...,...,...,...,...,...,...,...,...
111,00258801,카카오,035720,Y,[기재정정]주식매수선택권부여에관한신고,20230503000785,카카오,20230504,
112,00258801,카카오,035720,Y,[기재정정]주식매수선택권부여에관한신고,20230503000778,카카오,20230504,
113,00519252,THE E&M,089230,K,주요사항보고서(전환사채권발행결정),20230503000781,THE E&M,20230504,
114,00107598,남양유업,003920,Y,[기재정정]주요사항보고서(유상증자결정),20230503000780,남양유업,20230504,


### 1-2 공시정보-기업개황

In [10]:
# 회사명에 "삼성전자"가 포함된 회사들에 대한 개황정보
dart.company('005930')

{'status': '000',
 'message': '정상',
 'corp_code': '00126380',
 'corp_name': '삼성전자(주)',
 'corp_name_eng': 'SAMSUNG ELECTRONICS CO,.LTD',
 'stock_name': '삼성전자',
 'stock_code': '005930',
 'ceo_nm': '한종희, 경계현',
 'corp_cls': 'Y',
 'jurir_no': '1301110006246',
 'bizr_no': '1248100998',
 'adres': '경기도 수원시 영통구  삼성로 129 (매탄동)',
 'hm_url': 'www.samsung.com/sec',
 'ir_url': '',
 'phn_no': '02-2255-0114',
 'fax_no': '031-200-7538',
 'induty_code': '264',
 'est_dt': '19690113',
 'acc_mt': '12'}

In [ ]:
['조건부자본증권미상환', '미등기임원보수', '회사채미상환', '단기사채미상환', '기업어음미상환', '채무증권발행', '사모자금사용', '공모자금사용', '임원전체보수승인', '임원전체보수유형', '주식총수', '회계감사', '감사용역', '회계감사용역계약', '사외이사', '신종자본증권미상환', '증자', '배당', '자기주식', '최대주주', '최대주주변동', '소액주주', '임원', '직원', '임원개인보수', '임원전체보수', '개인별보수', '타법인출자']

In [25]:
dart.finstate('삼성전자', 2021)

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,...,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_dt,bfefrmtrm_amount,ord,currency
0,20220308000798,11011,2021,00126380,005930,CFS,연결재무제표,BS,재무상태표,유동자산,...,2021.12.31 현재,"218,163,185,000,000",제 52 기,2020.12.31 현재,"198,215,579,000,000",제 51 기,2019.12.31 현재,"181,385,260,000,000",1,KRW
1,20220308000798,11011,2021,00126380,005930,CFS,연결재무제표,BS,재무상태표,비유동자산,...,2021.12.31 현재,"208,457,973,000,000",제 52 기,2020.12.31 현재,"180,020,139,000,000",제 51 기,2019.12.31 현재,"171,179,237,000,000",3,KRW
2,20220308000798,11011,2021,00126380,005930,CFS,연결재무제표,BS,재무상태표,자산총계,...,2021.12.31 현재,"426,621,158,000,000",제 52 기,2020.12.31 현재,"378,235,718,000,000",제 51 기,2019.12.31 현재,"352,564,497,000,000",5,KRW
3,20220308000798,11011,2021,00126380,005930,CFS,연결재무제표,BS,재무상태표,유동부채,...,2021.12.31 현재,"88,117,133,000,000",제 52 기,2020.12.31 현재,"75,604,351,000,000",제 51 기,2019.12.31 현재,"63,782,764,000,000",7,KRW
4,20220308000798,11011,2021,00126380,005930,CFS,연결재무제표,BS,재무상태표,비유동부채,...,2021.12.31 현재,"33,604,094,000,000",제 52 기,2020.12.31 현재,"26,683,351,000,000",제 51 기,2019.12.31 현재,"25,901,312,000,000",9,KRW
5,20220308000798,11011,2021,00126380,005930,CFS,연결재무제표,BS,재무상태표,부채총계,...,2021.12.31 현재,"121,721,227,000,000",제 52 기,2020.12.31 현재,"102,287,702,000,000",제 51 기,2019.12.31 현재,"89,684,076,000,000",11,KRW
6,20220308000798,11011,2021,00126380,005930,CFS,연결재무제표,BS,재무상태표,자본금,...,2021.12.31 현재,"897,514,000,000",제 52 기,2020.12.31 현재,"897,514,000,000",제 51 기,2019.12.31 현재,"897,514,000,000",13,KRW
7,20220308000798,11011,2021,00126380,005930,CFS,연결재무제표,BS,재무상태표,이익잉여금,...,2021.12.31 현재,"293,064,763,000,000",제 52 기,2020.12.31 현재,"271,068,211,000,000",제 51 기,2019.12.31 현재,"254,582,894,000,000",17,KRW
8,20220308000798,11011,2021,00126380,005930,CFS,연결재무제표,BS,재무상태표,자본총계,...,2021.12.31 현재,"304,899,931,000,000",제 52 기,2020.12.31 현재,"275,948,016,000,000",제 51 기,2019.12.31 현재,"262,880,421,000,000",21,KRW
9,20220308000798,11011,2021,00126380,005930,CFS,연결재무제표,IS,손익계산서,매출액,...,2021.01.01 ~ 2021.12.31,"279,604,799,000,000",제 52 기,2020.01.01 ~ 2020.12.31,"236,806,988,000,000",제 51 기,2019.01.01 ~ 2019.12.31,"230,400,881,000,000",23,KRW


In [21]:
dart.report('삼성전자', '임원전체보수', 2022)  # 미등기임원 보수현황

,rcept_no,corp_cls,corp_code,corp_name,nmpr,jan_avrg_mendng_am,mendng_totamt,rm
0,20230307000542,Y,00126380,삼성전자,11,"2,729,000,000","30,024,000,000",-


In [23]:
dart.report('삼성전자', '개인별보수', 2022)  # 미등기임원 보수현황

,rcept_no,corp_cls,corp_code,corp_name,inv_prm,frst_acqs_de,invstmnt_purps,frst_acqs_amount,bsis_blce_qy,bsis_blce_qota_rt,bsis_blce_acntbk_amount,incrs_dcrs_acqs_dsps_qy,incrs_dcrs_acqs_dsps_amount,incrs_dcrs_evl_lstmn,trmend_blce_qy,trmend_blce_qota_rt,trmend_blce_acntbk_amount,recent_bsns_year_fnnr_sttus_tot_assets,recent_bsns_year_fnnr_sttus_thstrm_ntpf
0,20230307000542,Y,00126380,삼성전자,삼성전기㈜,1977.01.01,경영참여,"250,000,000","17,693,000",23.7,"445,244,000,000",-,-,-,"17,693,000",23.7,"445,244,000,000","10,997,171,000,000","993,519,000,000"
1,20230307000542,Y,00126380,삼성전자,세메스㈜,1992.12.31,경영참여,"1,000,000,000","2,173,000",91.5,"71,906,000,000",-,-,-,"2,173,000",91.5,"71,906,000,000","2,065,558,000,000","185,762,000,000"
2,20230307000542,Y,00126380,삼성전자,SSA,1998.12.01,경영참여,"263,000,000","2,000,000",100.0,"32,622,000,000",-,-,-,"2,000,000",100.0,"32,622,000,000","573,688,000,000","27,116,000,000"
3,20230307000542,Y,00126380,삼성전자,SAPL,2006.07.01,경영참여,"793,000,000","877,133,000",100.0,"981,483,000,000",-,-,-,"877,133,000",100.0,"981,483,000,000","26,894,611,000,000","8,699,679,000,000"
4,20230307000542,Y,00126380,삼성전자,SME,2003.05.01,경영참여,"4,796,000,000","17,100,000",100.0,"7,644,000,000",-,-,-,"17,100,000",100.0,"7,644,000,000","541,255,000,000","19,997,000,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,20230307000542,Y,00126380,삼성전자,SERK,2007.07.19,경영참여,"4,600,000,000",0,100.0,"204,555,000,000",-,-,-,0,100.0,"204,555,000,000","1,149,970,000,000","-104,424,000,000"
140,20230307000542,Y,00126380,삼성전자,SEO,1997.01.01,경영참여,"120,000,000",0,100.0,"-10,043,000,000",-,-,-,0,100.0,"-10,043,000,000","1,960,000,000","-76,000,000"
141,20230307000542,Y,00126380,삼성전자,SGE,1995.05.25,경영참여,"827,000,000",0,100.0,"32,836,000,000",-,-,-,0,100.0,"32,836,000,000","1,031,216,000,000","23,024,000,000"
142,20230307000542,Y,00126380,삼성전자,SEEG,2012.07.09,경영참여,"23,000,000",0,0.1,"39,000,000",-,-,-,0,0.1,"39,000,000","1,055,570,000,000","90,110,000,000"


In [ ]:
# ==== 1-3. 공시정보 - 공시서류원본파일 ====
# 삼성전자 사업보고서 (2022년 반기사업보고서) 원문 텍스트
xml_text = dart.document('20220816001711')

# ==== 1-3. 공시정보 - 공시서류원본파일 전체 리스트 (사업보고서, 감사보고서) ====
# 삼성전자 사업보고서 (2022년 반기사업보고서) 원문 텍스트, 감사보고서
xml_text_list = dart.document_all('20220816001711')
xml_text = xml_text_list[0]

# ==== 1-4. 공시정보 - 고유번호 ====
# 종목코드로 고유번호 얻기
dart.find_corp_code('005930')

# 기업명으로 고유번호 얻기
dart.find_corp_code('삼성전자')


### 2. 사업보고서 ###
# 조회가능한 사업보고서의 항목: 
['조건부자본증권미상환', '미등기임원보수', '회사채미상환', '단기사채미상환', '기업어음미상환', '채무증권발행', '사모자금사용', '공모자금사용', '임원전체보수승인', '임원전체보수유형', '주식총수', '회계감사', '감사용역', '회계감사용역계약', '사외이사', '신종자본증권미상환', '증자', '배당', '자기주식', '최대주주', '최대주주변동', '소액주주', '임원', '직원', '임원개인보수', '임원전체보수', '개인별보수', '타법인출자']

dart.report('005930', '미등기임원보수', 2021)  # 미등기임원 보수현황
dart.report('005930', '증자', 2021) # 증자(감자) 현황
dart.report('005930', '배당', 2018)  # 배당에 관한 사항


### 3. 상장기업 재무정보 ###
# 상장법인(금융업 제외)의 주요계정과목(재무상태표, 손익계산서)

# 삼성전자 2021 재무제표 
dart.finstate('삼성전자', 2021) 

# 삼성전자 2021Q1 재무제표
dart.finstate('삼성전자', 2021, reprt_code='11013')

# 여러 상장법인(금융업 제외)의 주요계정과목(재무상태표, 손익계산서)
dart.finstate('00126380,00164779,00164742', 2021)
dart.finstate('005930, 000660, 005380', 2021)
dart.finstate('삼성전자, SK하이닉스, 현대자동차', 2021)

# 재무제표 XBRL 원본 파일 저장 (삼성전자 2021 사업보고서)
dart.finstate_xml('20220308000798', save_as='삼성전자_2021_사업보고서_XBRL.zip')

# 전체 재무제표 (삼성전자 2021 전체 재무제표)
dart.finstate_all('005930', 2021)

# XBRL 표준계정과목체계(계정과목)
dart.xbrl_taxonomy('BS1')


### 4. 지분공시 ###
# 대량보유 상황보고 (종목코드, 종목명, 고유번호 모두 지정 가능)
dart.major_shareholders('삼성전자')

# 임원ㆍ주요주주 소유보고 (종목코드, 종목명, 고유번호 모두 지정 가능)
dart.major_shareholders_exec('005930')

### 5. 주요사항보고서  ###
# dart.event(corp, event, start=None, end=None)
# 조회가능한 주요사항 항목: 
# ['부도발생', '영업정지', '회생절차', '해산사유', '유상증자', '무상증자', '유무상증자', '감자', '관리절차개시', '소송', '해외상장결정', '해외상장폐지결정', '해외상장', '해외상장폐지', '전환사채발행', '신주인수권부사채발행', '교환사채발행', '관리절차중단', '조건부자본증권발행', '자산양수도', '타법인증권양도', '유형자산양도', '유형자산양수', '타법인증권양수', '영업양도', '영업양수', '자기주식취득신탁계약해지', '자기주식취득신탁계약체결', '자기주식처분', '자기주식취득', '주식교환', '회사분할합병', '회사분할', '회사합병', '사채권양수', '사채권양도결정']

dart.event('052220', '영업정지', '2019') # iMBC(052220)
dart.event('라이트론', '회생절차', '2019') # 라이트론(069540)
dart.event('휴림네트웍스', '유상증자') # 휴림네트웍스(192410)
dart.event('미원상사', '무상증자') # 미원상사(084990)
dart.event('017810', '전환사채발행') # 풀무원(017810)
dart.event('키다리스튜디오', '신주인수권부사채발행') #  키다리스튜디오(020120) 
dart.event('이스트소프트', '교환사채발행') # 이스트소프트(047560) 


### 6. 증권신고서   ###
# dart.regstate(corp, key_word, start=None, end=None)
# 조회가능한 증권신고서 항목: ['주식의포괄적교환이전', '합병', '증권예탁증권', '채무증권', '지분증권', '분할']

dart.regstate('하림지주', '주식의포괄적교환이전')
dart.regstate('사조대림', '합병')
dart.regstate('에스앤케이', '증권예탁증권')
dart.regstate('BNK금융지주', '채무증권')
dart.regstate('금호전기', '지분증권')
dart.regstate('케이씨씨', '분할')

### 7. 확장 기능   ###
# 지정한 날짜의 공시목록 전체 (공시 시간 정보 포함)
dart.list_date_ex('2022-10-03')

# 하위 문서(sub_docs), 첨부 문서(attach_doc_list)
rcp_no = '20220308000798' 
dart.sub_docs(rcp_no) # 하위 문서 제목과 URL
dart.sub_docs(rcp_no, match='감사의견')  # 제목이 잘 매치되는 순서로 소트
dart.attach_doc_list(rcp_no) # 첨부 문서(i.e. 감사보고서) 제목과 URL
dart.attach_doc_list(rcp_no, match='감사의의견서') # 제목이 잘 매치되는 순서로 소트

# 첨부 파일 제목과 URL
dart.attach_files(rcp_no)

# URL을 지정한 파일로 다운로드
url = 'http://dart.fss.or.kr/pdf/download/excel.do?rcp_no=20220308000798&dcm_no=8446647&lang=ko'
fn = '[삼성전자]사업보고서_재무제표(2022.03.08)_ko.xls'
dart.download(url, fn)

# 첨부파일 모두 다운로드
for k in dart.attach_files('20221111000817'):
    url, fn = atts[k], k
    print(url, fn)
    dart.download(url, fn)